In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:

import nltk
import string
from nltk.stem import LancasterStemmer
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

Loading Data

In [3]:
#Training data
train_path = "/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt"
train_data = pd.read_csv(train_path, sep=':::', names=['Title', 'Genre', 'Description'], engine='python')

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54214 entries, 1 to 54214
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        54214 non-null  object
 1   Genre        54214 non-null  object
 2   Description  54214 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [5]:
train_data.head()

,Title,Genre,Description
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [6]:
#Test data
test_path = "/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt"
test_data = pd.read_csv(test_path, sep=':::', names=['Id', 'Title', 'Description'], engine='python')

In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54200 entries, 0 to 54199
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           54200 non-null  int64 
 1   Title        54200 non-null  object
 2   Description  54200 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [8]:
test_data.head()

,Id,Title,Description
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...


Preprocessing

In [9]:
train_data.isnull().sum()

Title          0
Genre          0
Description    0
dtype: int64

In [10]:
duplicated_count = train_data.duplicated().sum()
print("Number of duplicate entries: ", duplicated_count)

Number of duplicate entries:  0


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [13]:
newStopWords = ['u','go','got','via','or','ur','us','in','i','let','the','to','is','amp','make','one','day','days','get']
stopwords.extend(newStopWords)

In [14]:
def data_processing(text):
    text= text.lower()
    text = re.sub('<br />', '', text)
    text = re.sub(r"https\S+|www\S+|http\S+", '', text, flags = re.MULTILINE)
    text = re.sub(r'\@w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stopwords]
    return " ".join(filtered_text)

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
train_data['Clean_text'] = train_data['Description'].apply(data_processing)
test_data['Clean_text'] = test_data['Description'].apply(data_processing)

In [17]:
train_data.head(5)

,Title,Genre,Description,Clean_text
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,listening conversation doctor parents 10yearol...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,brother sister past incestuous relationship cu...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,bus empties students field trip museum natural...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,help unemployed father ends meet edith twin si...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,films title refers unrecovered bodies ground z...


In [18]:
#tokenization
train_data['Clean_text'] = train_data['Clean_text'].apply(lambda x: str(x) if type(x) != str else x)
train_data['Clean_text'] = train_data['Clean_text'].apply(lambda x: x.split())
train_data.head()

,Title,Genre,Description,Clean_text
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,"[listening, conversation, doctor, parents, 10y..."
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,"[brother, sister, past, incestuous, relationsh..."
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,"[bus, empties, students, field, trip, museum, ..."
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,"[help, unemployed, father, ends, meet, edith, ..."
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,"[films, title, refers, unrecovered, bodies, gr..."


In [19]:
#tokenization
test_data['Clean_text'] = test_data['Clean_text'].apply(lambda x: str(x) if type(x) != str else x)
test_data['Clean_text'] = test_data['Clean_text'].apply(lambda x: x.split())
test_data.head()

,Id,Title,Description,Clean_text
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar...","[lr, brane, loves, life, car, apartment, job, ..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch...","[spain, march, 1964, quico, naughty, child, th..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...,"[year, life, albin, family, shepherds, north, ..."
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi...","[father, died, hasnt, spoken, brother, 10, yea..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...,"[known, internationally, martial, arts, supers..."


In [20]:
#stemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
train_data['Clean_text']= train_data['Clean_text'].apply(lambda x: [stemmer.stem(i) for i in x])
train_data.head()

,Title,Genre,Description,Clean_text
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,"[listen, convers, doctor, parent, 10yearold, o..."
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,"[brother, sister, past, incestu, relationship,..."
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,"[bus, empti, student, field, trip, museum, nat..."
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,"[help, unemploy, father, end, meet, edith, twi..."
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,"[film, titl, refer, unrecov, bodi, ground, zer..."


In [21]:
test_data['Clean_text']= train_data['Clean_text'].apply(lambda x: [stemmer.stem(i) for i in x])
test_data.head()

,Id,Title,Description,Clean_text
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar...",NaN
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch...","[listen, conver, doctor, parent, 10yearold, os..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...,"[brother, sister, past, incestu, relationship,..."
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi...","[bus, empti, student, field, trip, museum, nat..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...,"[help, unemploy, father, end, meet, edith, twi..."


Vectorization

In [23]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = tfidf_vectorizer.fit_transform(train_data['Clean_text'])

# Transform the test data
X_test = tfidf_vectorizer.transform(test_data['Clean_text'])

AttributeError: 'list' object has no attribute 'lower'